# YOLOv7
This notebook shows steps for training, testing and inference for training yolov7 or yolov7tiny models.

In [1]:
!nvidia-smi

Thu Jul 27 14:47:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Load all the files for yolov7 training:
- launch_yolov7.sh
- load_dataset.py
- test_mlflow.py
- train_mlflow.py
- data.yaml
- credentials.env
- hyp.scratch.custom.yaml
- hyp.scratch.tiny.yaml
- yolov7.yaml
- yolov7-tiny.yaml



If you want to run everything in one step, run the following command making sure to change the necessary parameters inside the "launch_yolov7.sh" file

In [ ]:
# !source launch_yolov7.sh

## Initialization

In [ ]:
# Set variables (Required)
%env dataset=Yolov7Dataset
%env model_type=yolov7-tiny # options: yolov7-tiny or yolov7
%env experiment_name=TechGo
%env run_tag=yolov7_tiny_640

# general hyperparameters (Required)
%env batch_size=16
%env epochs=1
%env img_size=640
%env data=data.yaml

# if training yolov7, complete next hyperparameters (Optional)
%env cfg_yolov7=yolov7.yaml
%env hyp_yolov7=hyp.scratch.custom.yaml

# if training yolov7-tiny, complete next hyperparameters (Optional)
%env cfg_yolov7_tiny=yolov7-tiny.yaml
%env hyp_yolov7_tiny=hyp.scratch.tiny.yaml

1. Download the official YOLOv7 repo

In [ ]:
%%bash
# Download the official YOLOv7 repo
if [ ! -d  "yolov7" ]; then
    git clone "https://github.com/WongKinYiu/yolov7.git"
else
    echo "The repository already exists in the target directory."
fi

2. Install the necessary dependencies

In [ ]:
# Install required packages
!pip install -r yolov7/requirements.txt
!pip install mlflow
!pip install boto3
!pip install python-dotenv

3. Move the training files with mlflow inside repo

In [5]:
%%bash
# Move modified traninig files
if [ -e "train_mlflow.py" ]; then
    mv train_mlflow.py yolov7
fi
if [ -e "test_mlflow.py" ]; then
    mv test_mlflow.py yolov7
fi

## Load dataset

In [ ]:
%%bash
if [ ! -d $dataset ]; then
    python3 load_dataset.py \
            --bucket_name elvis-s3-mlflow \
            --dataset_path dataset/$dataset.zip \
            --save_dir $dataset.zip

    unzip $dataset.zip -d .
    rm $dataset.zip

else
    echo "The dataset already exist"
fi

In [ ]:
import os
dataset_var = os.environ["dataset"]
num_training_images = len(os.listdir(f"/content/{dataset_var}/train/images"))
num_training_labels = len(os.listdir(f"/content/{dataset_var}/train/labels"))
num_valid_images = len(os.listdir(f"/content/{dataset_var}/valid/images"))
num_valid_labels = len(os.listdir(f"/content/{dataset_var}/valid/labels"))
num_testing_images = len(os.listdir(f"/content/{dataset_var}/test/images"))
num_testing_labels = len(os.listdir(f"/content/{dataset_var}/test/labels"))
print("Number of images in the train set. {}".format(num_training_images))
print("Number of labels in the train set. {}".format(num_training_labels))
print("Number of images in the valid set. {}".format(num_valid_images))
print("Number of labels in the valid set. {}".format(num_valid_labels))
print("Number of images in the test set. {}".format(num_testing_images))
print("Number of labels in the test set. {}".format(num_testing_labels))

## Training

Yolov7 training

In [ ]:
# Download weights
%%bash
cd yolov7
wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt
cd ..

In [ ]:
# Train an test yolov7 model
!python3 yolov7/train_mlflow.py \
        --device 0 --batch-size $batch_size --epochs $epochs --img $img_size \
        --data $data \
        --cfg $cfg_yolov7 \
        --weights 'yolov7/yolov7_training.pt' \
        --name $run_tag \
        --hyp $hyp_yolov7 \
        --experiment-name $experiment_name

Yolov7_tiny training

In [ ]:
# Download weights
%%bash
cd yolov7
wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt
cd ..

In [ ]:
# Train and test yolov7tiny model
!python3 yolov7/train_mlflow.py \
        --device 0 --batch-size $batch_size --epochs $epochs --img $img_size \
        --data $data \
        --cfg $cfg_yolov7_tiny \
        --weights 'yolov7/yolov7-tiny.pt' \
        --name $run_tag \
        --hyp $hyp_yolov7_tiny \
        --experiment-name $experiment_name

## Test

In [10]:
# %env run_tag=yolov7_tiny_640

In [ ]:
!python yolov7/test.py --weights runs/train/$run_tag/weights/best.pt --task test --data data.yaml --batch-size 16 --img-size 640

## Inference

In [ ]:
!python yolov7/detect.py --weights runs/train/$run_tag/weights/best.pt --conf 0.4 --img-size 640 --source /content/$dataset/test/images/230217_cam60_file01_frame100.jpg